In [12]:
import emd, os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import hilbert
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

sys.path.append(os.path.abspath(os.path.join('..', 'Features')))
import FeatureExtraction

In [13]:
def calculate_upper_envelope_hilbert(hilbert_signal):
    # Calculate magnitude of the Hilbert transformed signal
    envelope = np.abs(hilbert_signal)
    
    # Find local maxima
    maxima_indices, _ = find_peaks(envelope)
    maxima_values = envelope[maxima_indices]
    
    # Extend range of indices to cover entire signal length
    maxima_indices_extended = np.concatenate(([0], maxima_indices, [len(envelope) - 1]))
    maxima_values_extended = np.concatenate(([0], maxima_values, [0]))
    
    # Interpolate between local maxima
    interpolate_func = interp1d(maxima_indices_extended, maxima_values_extended, kind='cubic', bounds_error=False, fill_value=0)
    
    # Evaluate the interpolated function over the entire range
    upper_envelope = interpolate_func(np.arange(len(envelope)))
    
    return upper_envelope

In [14]:
def hilbert_spec_features(imfs) -> list:
    hilbert_spectrum_features = []

    for imf in imfs:
        analytic_signal = hilbert(imf)
        instantaneous_phase = np.unwrap(np.angle(analytic_signal))
        instantaneous_frequency = np.diff(instantaneous_phase) / (2.0 * np.pi) * 20000

        # Extract relevant features from the Hilbert spectrum
        # Example: peak frequency, energy distribution, etc.
        peak_frequency = np.max(instantaneous_frequency)
        energy_distribution = np.sum(np.abs(analytic_signal)**2)

        # Append features to the feature vector
        hilbert_spectrum_features.append([peak_frequency, energy_distribution])

    return hilbert_spectrum_features


In [19]:
%%time
directory = "../data/IMS/2nd_test/2nd_test"
window = np.hanning(20480)

print(directory)

for fileName in os.listdir(directory):
    
    filePath = os.path.join(directory, fileName)
    col_names = ['B1', 'B2', 'B3', 'B4']

    data = pd.read_csv(filePath, sep='\t', names=col_names)

    bearing1 = np.array(data['B1'])
    bearing2 = np.array(data['B2'])
    bearing3 = np.array(data['B3'])
    bearing4 = np.array(data['B4'])

    imf_bearing1 = emd.sift.mask_sift(bearing1, max_imfs=5)
    imf_bearing2 = emd.sift.mask_sift(bearing2, max_imfs=5)
    imf_bearing3 = emd.sift.mask_sift(bearing3, max_imfs=5)
    imf_bearing4 = emd.sift.mask_sift(bearing4, max_imfs=5)

    imf1_bearing1 = imf_bearing1[:, 0]
    imf1_bearing2 = imf_bearing2[:, 0]
    imf1_bearing3 = imf_bearing3[:, 0]
    imf1_bearing4 = imf_bearing4[:, 0]

    hilbert_imf1_bearing1 = hilbert(imf1_bearing1)
    hilbert_imf1_bearing2 = hilbert(imf1_bearing2)
    hilbert_imf1_bearing3 = hilbert(imf1_bearing3)
    hilbert_imf1_bearing4 = hilbert(imf1_bearing4)

    test = hilbert_spec_features(imf_bearing1)

    envelope_imf1_bearing1 = calculate_upper_envelope_hilbert(hilbert_imf1_bearing1)
    envelope_imf1_bearing2 = calculate_upper_envelope_hilbert(hilbert_imf1_bearing2)
    envelope_imf1_bearing3 = calculate_upper_envelope_hilbert(hilbert_imf1_bearing3)
    envelope_imf1_bearing4 = calculate_upper_envelope_hilbert(hilbert_imf1_bearing4)

    windowed_envelope_bearing1 = window * envelope_imf1_bearing1
    windowed_envelope_bearing2 = window * envelope_imf1_bearing2
    windowed_envelope_bearing3 = window * envelope_imf1_bearing3
    windowed_envelope_bearing4 = window * envelope_imf1_bearing4

    envelope_fourier1 = fft(windowed_envelope_bearing1)
    envelope_fourier2 = fft(windowed_envelope_bearing2)
    envelope_fourier3 = fft(windowed_envelope_bearing3)
    envelope_fourier4 = fft(windowed_envelope_bearing4)

    # plt.plot(envelope_fourier1)
    

../data/IMS/2nd_test/2nd_test


IsADirectoryError: [Errno 21] Is a directory: '../data/IMS/2nd_test/2nd_test/.ipynb_checkpoints'

In [16]:
print(test)

NameError: name 'test' is not defined